# Analysis

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import os
import sys 

sys.path.append(os.path.join(os.getcwd(), '..'))
import facemap_tools.process_tools

### Looking into pupil area plotted 